In [1]:
import copy
import sqlite3
import numpy as np
import pandas as pd
from rdkit import Chem

#### 1. Extracting data from Database

In [2]:
def call_my_query(db_file, my_query):
    ## connect to the SQLIte database
    my_connection = sqlite3.connect(db_file)

    ## create a cursor object
    my_cursor = my_connection.cursor()

    ## excute the query
    my_cursor.execute(my_query)

    ## fetch all the rows
    rows = my_cursor.fetchall()
    
    ## export the results
    data_list = [row for row in rows]

    my_connection.close()
    return data_list

def extract_tables(db_file, table_name):
    ## extract table data from SQLite DB
    my_query_colName = f"PRAGMA table_info({table_name})"
    colName_list = call_my_query(db_file, my_query_colName)

    my_query_data = f"SELECT * FROM {table_name}"
    data_list = call_my_query(db_file, my_query_data)

    ## clean up data
    dataDict = {}
    for row_tuple in data_list:
        idx = row_tuple[0]
        dataDict[idx] = {}

        for col in colName_list:
            colIdx, colName = col[0], col[1]
            dataDict[idx][colName] = row_tuple[colIdx]
    return dataDict

In [3]:
db_file = './results/Compounds_All.mmpdb'
dataDict_tables = {}

for table_name in ["pair", "compound", "compound_property", "property_name", "constant_smiles",
                   "rule", "rule_smiles", "rule_environment", "rule_environment_statistics", "environment_fingerprint"]:
    dataDict_table = extract_tables(db_file, table_name)
    dataDict_tables[table_name] = pd.DataFrame.from_dict(dataDict_table).T
    # print(table_name)

In [4]:
dataDict_tables["property_name"]

,id,name
0,0,F%_Rat
1,1,EstFa_Rat
2,2,permeability
3,3,efflux
4,4,hERG_IC50
5,5,hERG_mixedIC50
6,6,logD_CDD


In [5]:
dataTable_rule_env_stats = copy.deepcopy(dataDict_tables["rule_environment_statistics"])
print(dataTable_rule_env_stats.shape)
dataTable_rule_env_stats.head(3)

(1548581, 15)


,id,rule_environment_id,property_name_id,count,avg,std,kurtosis,skewness,min,q1,median,q3,max,paired_t,p_value
1,1.0,343.0,0.0,3.0,6.673333,3.715432,-1.500000,0.374010,4.240000,4.3875,4.830000,9.42,10.95,3.110958,0.089649
2,2.0,349.0,0.0,34.0,3.912390,8.627412,11.345179,2.765673,-10.023889,0.1256,1.478167,6.92,43.55,2.644241,0.012434
3,3.0,433.0,0.0,1.0,6.530000,NaN,NaN,NaN,6.530000,6.5300,6.530000,6.53,6.53,NaN,NaN


In [6]:
dataTable_rule_env_stats = copy.deepcopy(dataDict_tables["rule_environment_statistics"])
dataTable_rule_env_stats.drop(columns=['id', 'kurtosis','skewness', 'paired_t', 'p_value', 'q1', 'median', 'q3'], inplace=True)
dataTable_rule_env_stats = dataTable_rule_env_stats.merge(dataDict_tables["property_name"], left_on='property_name_id', right_on='id')
print(dataTable_rule_env_stats.shape)
dataTable_rule_env_stats.head(3)

(1548581, 9)


,rule_environment_id,property_name_id,count,avg,std,min,max,id,name
0,343.0,0.0,3.0,6.673333,3.715432,4.240000,10.95,0,F%_Rat
1,349.0,0.0,34.0,3.912390,8.627412,-10.023889,43.55,0,F%_Rat
2,433.0,0.0,1.0,6.530000,NaN,6.530000,6.53,0,F%_Rat


In [7]:
dataTable_rule_env_fp = copy.deepcopy(dataDict_tables["environment_fingerprint"])
dataTable_rule_env_fp.rename(columns={'id':'environment_fingerprint_id', 
                                      'pseudosmiles':'rule_env_fp_pseudosmiles',
                                      'smarts':'rule_env_fp_smarts', 
                                      'parent_smarts':'rule_env_fp_parent_smarts'})
print(dataTable_rule_env_fp.shape)
dataTable_rule_env_fp.head(3)

(29023, 4)


,id,smarts,pseudosmiles,parent_smarts
1,1,[#0;X1;H0;+0;!R:1],[*:1](~*),
2,2,[#0;X1;H0;+0;!R:1]-[C;X3;H1;+0;!R],[*:1]-[CH](~*),[#0;X1;H0;+0;!R:1]
3,3,[#0;X1;H0;+0;!R:1]-[C;X3;H1;+0;!R]=[#6;X3;H0;+...,[*:1]-[CH]=[#6](~*)(~*),[#0;X1;H0;+0;!R:1]-[C;X3;H1;+0;!R]


#### 2. clean up data

In [ ]:
## -------------------- clean up rule table & merge rule smiles --------------------
dataTable_rules = copy.deepcopy(dataDict_tables["rule"])
dataTable_rules.rename(columns={'id':'rule_id'},  inplace=True)

## from
dataTable_rules = dataTable_rules.merge(dataDict_tables["rule_smiles"], left_on=['from_smiles_id'], right_on=['id'])
dataTable_rules.drop(columns=['id', 'num_heavies'], inplace=True)
dataTable_rules.rename(columns={'smiles':'from_smiles'}, inplace=True)
dataTable_rules.head(3)

## to 
dataTable_rules = dataTable_rules.merge(dataDict_tables["rule_smiles"], left_on=['to_smiles_id'], right_on=['id'])
dataTable_rules.drop(columns=['id', 'num_heavies'], inplace=True)
dataTable_rules.rename(columns={'smiles':'to_smiles'}, inplace=True)

## -------------------- merge rule table and rule env table --------------------
dataTable_rule_env = copy.deepcopy(dataDict_tables["rule_environment"])
dataTable_rule_env.rename(columns={'id':'rule_environment_id', 'radius':'rule_env_radius', 'num_pairs':'rule_env_num_pairs'},  inplace=True)
dataTable_rule_env = dataTable_rule_env.merge(dataTable_rules, on='rule_id')

## -------------------- merge rule env table and rule_env_stats info --------------------
## clean up rule-env-stats table
dataTable_rule_env_stats = copy.deepcopy(dataDict_tables["rule_environment_statistics"])
dataTable_rule_env_stats.drop(columns=['id', 'kurtosis','skewness', 'paired_t', 'p_value', 'q1', 'median', 'q3'], inplace=True)
dataTable_rule_env_stats = dataTable_rule_env_stats.merge(dataDict_tables["property_name"], left_on='property_name_id', right_on='id')

## merge
dataTable_rule_env = dataTable_rule_env.merge(dataTable_rule_env_stats, left_on=['rule_environment_id'], right_on=['rule_environment_id'])


## -------------------- merge rule env table and rule_env_fp info --------------------
dataTable_rule_env_fp = copy.deepcopy(dataDict_tables["environment_fingerprint"])
dataTable_rule_env_fp.rename(columns={'id':'environment_fingerprint_id', 
                                      'pseudosmiles':'rule_env_fp_pseudosmiles',
                                      'smarts':'rule_env_fp_smarts', 
                                      'parent_smarts':'rule_env_fp_parent_smarts'}, inplace=True)

dataTable_rule_env = dataTable_rule_env.merge(dataTable_rule_env_fp, on=['environment_fingerprint_id'])
# dataTable_rule_env.drop(columns=['id'], inplace=True)    #, 'smarts', 'parent_smarts'
dataTable_rule_env.rename(columns={'environment_fingerprint_id':'rule_env_fingerprint_id',
                                   'pseudosmiles':'rule_env_fp_pseudosmiles', 
                                   'smarts':'rule_env_fp_smarts', 
                                   'parent_smarts':'rule_env_fp_parent_smarts', }, inplace=True)


# cols_in_order = ['rule_id', 'from_smiles_id', 'from_smiles', 'to_smiles_id', 'to_smiles', 
#                  'rule_environment_id', 'rule_env_num_pairs', 'rule_env_radius', 'rule_env_fingerprint_id', 
#                  'rule_env_fp_pseudosmiles', 'rule_env_fp_smarts', 'rule_env_fp_parent_smarts']
# dataTable_rule_env = dataTable_rule_env[cols_in_order]

dataTable_rule_env

In [ ]:
dataDict_tables["rule_smiles"]

In [ ]:
dataTable = copy.deepcopy(dataDict_tables["pair"])
dataTable.head(3)

## ------------------- add compound structure & property data -------------------
table_merge = dataDict_tables["compound"]

## compound-1 (from)
dataTable = dataTable.merge(table_merge, left_on=['compound1_id'], right_on=['id'])
dataTable.drop(columns=['id_y', 'clean_smiles', 'clean_num_heavies'], inplace=True)
dataTable.rename(columns={'id_x':'id', 'public_id':'KT_number_1', 'input_smiles':'smiles_1'}, inplace=True)

## compound-2 (to)
dataTable = dataTable.merge(table_merge, left_on=['compound2_id'], right_on=['id'])
dataTable.drop(columns=['id_y', 'clean_smiles', 'clean_num_heavies'], inplace=True)
dataTable.rename(columns={'id_x':'id', 'public_id':'KT_number_2', 'input_smiles':'smiles_2'}, inplace=True)

## ------------------- add compound prop data -------------------
table_merge = dataDict_tables["compound_property"]

## compound-1 (from)
dataTable = dataTable.merge(table_merge, left_on=['compound1_id'], right_on=['compound_id'])
dataTable.drop(columns=['id_y', 'compound_id'], inplace=True)
dataTable.rename(columns={'id_x':'id', 'value':'property_values_1'}, inplace=True)

## compound-2 (to)
dataTable = dataTable.merge(table_merge, left_on=['compound2_id', 'property_name_id'], right_on=['compound_id', 'property_name_id'])
dataTable.drop(columns=['id_y', 'compound_id'], inplace=True)
dataTable.rename(columns={'id_x':'id', 'value':'property_values_2'}, inplace=True)

## add property name
table_merge = dataDict_tables["property_name"]
dataTable = dataTable.merge(table_merge, left_on=['property_name_id'], right_on=['id'])
dataTable.drop(columns=['id_y'], inplace=True)
dataTable.rename(columns={'id_x':'id', 'name':'property_name'}, inplace=True)

## ------------------- add constant pieces data of the match pair -------------------
table_merge = dataDict_tables["constant_smiles"]
dataTable = dataTable.merge(table_merge, left_on=['constant_id'], right_on=['id'])
dataTable.drop(columns=['id_y'], inplace=True)
dataTable.rename(columns={'id_x':'id', 'smiles':'constant_smiles'}, inplace=True)

## ------------------- add rule env data -------------------
table_merge = dataDict_tables["rule_environment"]
dataTable = dataTable.merge(table_merge, left_on=['rule_environment_id'], right_on=['id'])
dataTable.drop(columns=['id_y'], inplace=True)
dataTable.rename(columns={'id_x':'id', 'radius':'rule_env_radius', 'num_pairs':'rule_env_num_pairs'}, inplace=True)

## ------------------- add rule info -------------------
table_merge = dataDict_tables["rule"]
dataTable = dataTable.merge(table_merge, left_on=['rule_id'], right_on=['id'])
dataTable.drop(columns=['id_y'], inplace=True)    #'rule_id'
dataTable.rename(columns={'id_x':'id'}, inplace=True)

table_merge = dataDict_tables["rule_smiles"]
dataTable = dataTable.merge(table_merge, left_on=['from_smiles_id'], right_on=['id'])
dataTable.drop(columns=['id_y', 'from_smiles_id', 'num_heavies'], inplace=True)    #'num_heavies'
dataTable.rename(columns={'id_x':'id', 'smiles':'rule_from_smiles'}, inplace=True)

table_merge = dataDict_tables["rule_smiles"]
dataTable = dataTable.merge(table_merge, left_on=['to_smiles_id'], right_on=['id'])
dataTable.drop(columns=['id_y', 'to_smiles_id', 'num_heavies'], inplace=True)
dataTable.rename(columns={'id_x':'id', 'smiles':'rule_to_smiles'}, inplace=True)

## ------------------- add rule env stats -------------------
table_merge = dataDict_tables["rule_environment_statistics"]
dataTable = dataTable.merge(table_merge, 
                            left_on=['rule_environment_id', 'property_name_id'], 
                            right_on=['rule_environment_id', 'property_name_id'])

drop_cols = ['kurtosis', 'skewness', 'paired_t', 'p_value', 'q1', 'q3', 'median', 'std']
dataTable.drop(columns=['id_y']+drop_cols, inplace=True)
dataTable.rename(columns={'id_x':'id', 'count':'rule_env_count', 'avg':'rule_env_avg', 
                          'min':'rule_env_min', 'max':'rule_env_max'}, inplace=True)

## ------------------- add rule env environment_fingerprint data -------------------
table_merge = dataDict_tables["environment_fingerprint"]
## to be added

## ------------------- remove useless cols -------------------
dataTable.drop(columns=['id', 'compound1_id', 'compound2_id', 'constant_id', 'rule_environment_id', 'property_name_id'], inplace=True)
print(dataTable.shape)
dataTable.head(3)

In [ ]:
def GeneratePairID(row, col_mol_id_1='KT_number_1', col_mol_id_2='KT_number_2'):
    mol_id_1 = row[col_mol_id_1]
    mol_id_2 = row[col_mol_id_2]
    pair_id = str(mol_id_1) + '=>' + str(mol_id_2)
    
    mol_id_1_num = int(str(mol_id_1).split('-')[1])
    mol_id_2_num = int(str(mol_id_2).split('-')[1])
    pair_couple = (np.min([mol_id_1_num, mol_id_2_num]), np.max([mol_id_1_num, mol_id_2_num]))
    return pd.Series([pair_id, pair_couple])

dataTable[['Pair_id', 'PairInfo']] = dataTable.apply(lambda row: GeneratePairID(row, col_mol_id_1='KT_number_1', col_mol_id_2='KT_number_2'), axis=1)
print(dataTable.shape)

################################################################################################
def calculate_heavy_atoms(molecule_smiles):
    try:
        mol = Chem.MolFromSmiles(molecule_smiles)
        num_heavy_atoms = mol.GetNumHeavyAtoms()
    except Exception as e:
        print('Error', e)
        num_heavy_atoms = np.nan
    return num_heavy_atoms

dataTable['constant_size'] = dataTable['constant_smiles'].apply(calculate_heavy_atoms)
dataTable.sort_values(by=['PairInfo', 'Pair_id', 'rule_env_radius', 'constant_size'], ascending=[True, True, True, False], inplace=True)
print(dataTable.shape)

################################################################################################

In [ ]:
def calculate_heavy_atoms(molecule_smiles):
    try:
        mol = Chem.MolFromSmiles(molecule_smiles)
        num_heavy_atoms = mol.GetNumHeavyAtoms()
    except Exception as e:
        print('Error', e)
        num_heavy_atoms = np.nan
    return num_heavy_atoms

dataTable['constant_size'] = dataTable['constant_smiles'].apply(calculate_heavy_atoms)
dataTable.sort_values(by=['PairInfo', 'Pair_id', 'rule_env_radius', 'constant_size'], ascending=[True, True, True, False], inplace=True)
# dataTable.to_csv(f'./results/Compounds_All_4_informatics.csv', index=False)
dataTable.head(3)

#### 3. removed the "duplicated" rows

In [ ]:
dataTable.sort_values(by=['PairInfo', 'rule_env_radius', 'constant_size'], ascending=[True, True, False], inplace=True)
dataTable_rmDup = dataTable.drop_duplicates(subset=['PairInfo', 'property_name'], keep='first', inplace=False)
print(dataTable_rmDup.shape)
dataTable_rmDup.head(3)

#### append symetric rows

In [ ]:
rename_symetric_dict = {
    'KT_number_1': 'KT_number_2',
    'smiles_1': 'smiles_2',
    'KT_number_2': 'KT_number_1',
    'smiles_2': 'smiles_1',
    'property_values_1': 'property_values_2',
    'property_values_2': 'property_values_1', 
    'rule_from_smiles': 'rule_to_smiles',
    'rule_to_smiles': 'rule_from_smiles'}
dataTable_rmDup_symetric = dataTable_rmDup.rename(columns=rename_symetric_dict, inplace=False)
dataTable_rmDup_symetric['Pair_id'] = dataTable_rmDup_symetric['KT_number_1'] + '=>' + dataTable_rmDup_symetric['KT_number_2']
for col in ['rule_env_avg', 'rule_env_min', 'rule_env_max']:
    dataTable_rmDup_symetric[col] = dataTable_rmDup_symetric[col] * -1
dataTable_rmDup_symetric.head(3)

In [ ]:
dataTable_rmDup_all = dataTable_rmDup._append(dataTable_rmDup_symetric, ignore_index=True)
dataTable_rmDup_all['rule_env_min'] = dataTable_rmDup_all['rule_env_min'].apply(lambda x:round(x, 2)).astype('str')
dataTable_rmDup_all['rule_env_max'] = dataTable_rmDup_all['rule_env_max'].apply(lambda x:round(x, 2)).astype('str')
dataTable_rmDup_all['rule_env_range'] = '('+ dataTable_rmDup_all['rule_env_min'] + ',' + dataTable_rmDup_all['rule_env_max'] +')'

In [ ]:
dataTable_rmDup_all.sort_values(by=['PairInfo', 'property_name', 'Pair_id'], ascending=[True, True, True], inplace=True)
dataTable_rmDup_all = dataTable_rmDup_all.reset_index(drop=True)
# dataTable_rmDup_all.to_csv(f'./results/Compounds_All_4_informatics_rmDups.csv', index=False)
dataTable_rmDup_all